In [1]:
import csv

# open and read the csv file
f = open("quiz4_routing.csv")
csvfile = csv.DictReader(f, delimiter=',')
headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()

# define the set of routes
Routes = headers[:]
Routes.remove('Distribution Center')
Routes.remove('Demand (lbs)')

# read the parameters
Demand = {}        # Demand[j] is the demand for distribution center j
IsInRoute = {}     # IsInRoute[i,j] = 1 if route i includes distribution center j
Length = {}        # Length[i] is the length of route i
for row in table:
    dc = row['Distribution Center']
    if dc != 'n/a':
        Demand[dc] = float(row['Demand (lbs)'])
        for r in Routes:
            if row[r] != '':
                IsInRoute[(r,dc)] = int(row[r])
    else:
        for r in Routes:
            Length[r] = float(row[r])
DCs = Demand.keys()
Capacity = 30000  # Truck capacity

In [2]:
from docplex.mp.model import Model
mdl = Model()

In [3]:
# Define decision variables
select = mdl.binary_var_dict(Routes, name='route')

In [4]:
# Objective function
mdl.minimize(mdl.sum(Length[r] * select[r] for r in Routes))

In [5]:
# Constraints
# Define the constraints
for dc in DCs:
    mdl.add_constraint(mdl.sum(IsInRoute.get((r, dc), 0) * select[r] for r in Routes) >= 1, f'Demand_{dc}')

In [6]:
# Constraints: Truck capacity must not be exceeded in any route
for r in Routes:
    mdl.add_constraint(mdl.sum(Demand[dc] for dc in DCs if (r, dc) in IsInRoute) <= Capacity)

In [7]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0.063,status='integer optimal solution')

In [8]:
mdl.print_solution()

objective: 7922.000
status: OPTIMAL_SOLUTION(2)
  route_5=1
  route_6=1
  route_9=1
  route_17=1
  route_24=1
  route_26=1
  route_27=1


In [9]:
# Solve the model
solution = mdl.solve()

# Output the solution
if solution:
    print("Total distance:", solution.objective_value)
    print("Routes used:")
    for r in Routes:
        if select[r].solution_value > 0.5:
            print(f"Route {r} with length {Length[r]} and centers {[dc for dc in DCs if (r, dc) in IsInRoute]}")
else:
    print("No solution found")

Total distance: 7922.0
Routes used:
Route 5 with length 317.0 and centers ['5']
Route 6 with length 1123.0 and centers ['6']
Route 9 with length 1061.0 and centers ['9']
Route 17 with length 1560.0 and centers ['11', '12', '15']
Route 24 with length 1688.0 and centers ['3', '4', '7', '8']
Route 26 with length 1022.0 and centers ['2', '13']
Route 27 with length 1151.0 and centers ['1', '10', '14']
